In [161]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Superheros").getOrCreate()

In [162]:
# read csv
graph = spark.read.text("../data/marvel/Marvel+Graph")

graph.show(5)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
+--------------------+
only showing top 5 rows



Breadth first search. Algorithm that searches a graph to obtain the distance between any given pair of vertices.


In [163]:
curated_graph = graph.withColumn(
    "value",
    F.split(F.trim(graph["value"]), " "),  # split space separated string to array
)

curated_graph = curated_graph.withColumn(
    "vertex_id", curated_graph["value"][0]
).withColumn("num_of_connections", F.size(curated_graph["value"]) - 1)

curated_graph = curated_graph.withColumn(
    "connections",
    F.slice(curated_graph["value"], 2, curated_graph["num_of_connections"]),
)

curated_graph.show(5)

+--------------------+---------+------------------+--------------------+
|               value|vertex_id|num_of_connections|         connections|
+--------------------+---------+------------------+--------------------+
|[5988, 748, 1722,...|     5988|                48|[748, 1722, 3752,...|
|[5989, 4080, 4264...|     5989|                40|[4080, 4264, 4446...|
|[5982, 217, 595, ...|     5982|                42|[217, 595, 1194, ...|
|[5983, 1165, 3836...|     5983|                14|[1165, 3836, 4361...|
|[5980, 2731, 3712...|     5980|                24|[2731, 3712, 1587...|
+--------------------+---------+------------------+--------------------+
only showing top 5 rows



In [164]:
def flatten(xss):
    return [x for xs in xss for x in xs]


# Get degrees of separation
initial_vertex = 5988

# initialize distance to -1
df = curated_graph.withColumn("distance", F.lit(-1))

vertices = [initial_vertex]
distance = 0
max_iterations = 8
num_unsolved_vertices = df.count()

for i in range(max_iterations):
    # set distance of the vertices
    df = df.withColumn(
        "distance",
        F.when(
            (df["vertex_id"].isin(vertices)) & (df["distance"] == -1), distance
        ).otherwise(df["distance"]),
    )

    # get a list of all the neighbors of the vertices
    neighbors_rows = (
        df.filter(df["distance"] == distance).select("connections").collect()
    )
    neighbors = [row["connections"] for row in neighbors_rows]
    neighbors = flatten(neighbors)

    # Exit loop i there are no remaining unsolved vertices
    prev_unsolved_vertices = num_unsolved_vertices
    num_unsolved_vertices = df.filter(df["distance"] == -1).count()
    if num_unsolved_vertices == 0 or prev_unsolved_vertices == num_unsolved_vertices:
        break

    # For the next iteration, the distance is increased by one
    # and the vertices will be the neighbors of the current iteration
    vertices = neighbors
    distance = distance + 1


df.show(5)

+--------------------+---------+------------------+--------------------+--------+
|               value|vertex_id|num_of_connections|         connections|distance|
+--------------------+---------+------------------+--------------------+--------+
|[5988, 748, 1722,...|     5988|                48|[748, 1722, 3752,...|       0|
|[5989, 4080, 4264...|     5989|                40|[4080, 4264, 4446...|       3|
|[5982, 217, 595, ...|     5982|                42|[217, 595, 1194, ...|       3|
|[5983, 1165, 3836...|     5983|                14|[1165, 3836, 4361...|       2|
|[5980, 2731, 3712...|     5980|                24|[2731, 3712, 1587...|       3|
+--------------------+---------+------------------+--------------------+--------+
only showing top 5 rows



In [167]:
df.groupBy("distance").count().show()

+--------+-----+
|distance|count|
+--------+-----+
|      -1|   37|
|       1|   49|
|       3| 5454|
|       4|  187|
|       2|  861|
|       0|    1|
+--------+-----+



In [166]:
spark.stop()